In [49]:
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import load_model
import skfuzzy as fuzz
import skfuzzy.control as ctrl
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score



In [50]:
# Load the saved features and labels
all_features_vgg16 = np.load('/Users/saikarthik/Desktop/scp/ga_vgg16/all_features_vgg16.npy')
all_labels_vgg16 = np.load('/Users/saikarthik/Desktop/scp/ga_vgg16/all_labels_vgg16.npy')

# Print shape to verify loading
print(f'Features shape: {all_features_vgg16.shape}')
print(f'Labels shape: {all_labels_vgg16.shape}')


Features shape: (6672, 25088)
Labels shape: (6672, 4)


In [51]:
all_features_vgg19 = np.load('/Users/saikarthik/Desktop/scp/ga_vgg19/all_features_vgg19.npy')
all_labels_vgg19 = np.load('/Users/saikarthik/Desktop/scp/ga_vgg19/all_labels_vgg19.npy')

# Print shape to verify loading
print(f'Features shape: {all_features_vgg16.shape}')
print(f'Labels shape: {all_labels_vgg16.shape}')

Features shape: (6672, 25088)
Labels shape: (6672, 4)


In [52]:
from tensorflow.keras.models import load_model

# Load the best VGG16 and VGG19 models
best_vgg16_model = load_model('/Users/saikarthik/Desktop/scp/ga_vgg16/best_vgg16.h5')
best_vgg19_model = load_model('/Users/saikarthik/Desktop/scp/ga_vgg19/best_vgg19.h5')


In [53]:
all_features_vgg16

array([[-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.0721586 , -0.04747448],
       [-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.11433829, -0.04747448],
       [-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.59788954, -0.04747448],
       ...,
       [-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.37254465, -0.04747448],
       [-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.59788954, -0.04747448],
       [-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.10403824, -0.04747448]], dtype=float32)

In [54]:
all_features_vgg19

array([[-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02,  1.3985383e+01],
       [-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02, -6.0834702e-02],
       [-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02, -6.0834702e-02],
       ...,
       [-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02, -6.0834702e-02],
       [-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02, -6.0834702e-02],
       [-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02, -6.0834702e-02]], dtype=float32)

In [55]:
all_labels_vgg16

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [56]:
all_labels_vgg19

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [57]:
num_classes = 4

final_predictions = []
sample_index = 0


In [58]:
# 1. Normalize predictions using fuzzy language
def normalize_probabilities(predictions):
    # Sum of fuzzy predictions
    total_fuzzy = np.sum(predictions)
    # Normalize using fuzzy concept (sum of fuzzy terms)
    return predictions / total_fuzzy

In [59]:


# 2. Decision-making using weighted average with fuzzy rules
def weighted_average_decision(prob_vgg16, prob_vgg19, weight_vgg16=0.6, weight_vgg19=0.4):
    # Normalize the fuzzy probabilities
    prob_vgg16 = normalize_probabilities(prob_vgg16)
    prob_vgg19 = normalize_probabilities(prob_vgg19)
    
    # Combine using fuzzy weights
    combined_fuzzy_prob = (weight_vgg16 * prob_vgg16) + (weight_vgg19 * prob_vgg19)
    
    # Make final decision (fuzzy maximum)
    final_class = np.argmax(combined_fuzzy_prob)
    return final_class


In [60]:


def adjust_weights(pred_vgg16, pred_vgg19, actual_label, weight_vgg16, weight_vgg19, threshold=0.1):
    predicted_class = weighted_average_decision(pred_vgg16, pred_vgg19, weight_vgg16, weight_vgg19)
    
    # Fuzzy check if predictions do not match the actual label
    if predicted_class != actual_label:
        print("Adjusting fuzzy weights.")
        
        # Adjust weight_vgg16 based on fuzzy rules
        if np.argmax(pred_vgg16) != actual_label:
            weight_vgg16 = max(0.1, weight_vgg16 - threshold)  # Fuzzy decrease in weight
        else:
            weight_vgg16 = min(1.0, weight_vgg16 + threshold)  # Fuzzy increase in weight

        # Adjust weight_vgg19 based on fuzzy rules
        if np.argmax(pred_vgg19) != actual_label:
            weight_vgg19 = max(0.1, weight_vgg19 - threshold)  # Fuzzy decrease in weight
        else:
            weight_vgg19 = min(1.0, weight_vgg19 + threshold)  # Fuzzy increase in weight
    
    return weight_vgg16, weight_vgg19, predicted_class



In [61]:

# 4. Adjust weights iteratively using fuzzy rules
def adjust_weights_with_iterations(pred_vgg16, pred_vgg19, actual_label, weight_vgg16, weight_vgg19, threshold=0.1, max_iterations=5):
    iteration = 0
    predicted_class = weighted_average_decision(pred_vgg16, pred_vgg19, weight_vgg16, weight_vgg19)
    
    while predicted_class != actual_label and iteration < max_iterations:
        print(f"Iteration {iteration + 1}: Prediction is incorrect.")
        
        # Adjust fuzzy weights on each iteration
        weight_vgg16, weight_vgg19, _ = adjust_weights(pred_vgg16, pred_vgg19, actual_label, weight_vgg16, weight_vgg19, threshold)
        
        predicted_class = weighted_average_decision(pred_vgg16, pred_vgg19, weight_vgg16, weight_vgg19)
        
        iteration += 1
    
    return weight_vgg16, weight_vgg19, predicted_class, iteration

In [ ]:

# 1 11 6100 1111 868
sample_index = 868

sample_vgg16 = all_features_vgg16[sample_index:sample_index+1] 
sample_vgg19 = all_features_vgg19[sample_index:sample_index+1]  
actual_label = np.argmax(all_labels_vgg16[sample_index])  

weight_vgg16 = 0.6
weight_vgg19 = 0.4

# Predicted fuzzy probabilities
pred_vgg16 = best_vgg16_model.predict(sample_vgg16)  
pred_vgg19 = best_vgg19_model.predict(sample_vgg19)  

# Print initial weights
print(f"Initial weights -> VGG16: {weight_vgg16}, VGG19: {weight_vgg19}")

# Predicted class using the weighted average decision
predicted_class = weighted_average_decision(pred_vgg16[0], pred_vgg19[0], weight_vgg16, weight_vgg19)

# Print initial prediction
if predicted_class == actual_label :
    print(f"Initial prediction: {predicted_class}, Actual label: {actual_label}")
    print("No need to update weights")
else :
    print(f"Initial prediction: {predicted_class}, Actual label: {actual_label}")
    
    # Adjust weights using fuzzy logic and iterate if necessary
    weight_vgg16, weight_vgg19, predicted_class, iterations = adjust_weights_with_iterations(
        pred_vgg16[0], pred_vgg19[0], actual_label, weight_vgg16, weight_vgg19)

    # Print the final result after adjustment
    print(f"Adjusted weights -> VGG16: {weight_vgg16}, VGG19: {weight_vgg19}")
    print(f"Predicted label after {iterations} iteration(s): {predicted_class}")


1/1 [==============================] - 0s 12ms/step
Initial weights -> VGG16: 0.6, VGG19: 0.4
Initial prediction: 0, Actual label: 3
Iteration 1: Prediction is incorrect.
Adjusting fuzzy weights.
Iteration 2: Prediction is incorrect.
Adjusting fuzzy weights.
Adjusted weights -> VGG16: 0.4, VGG19: 0.6
Predicted label after 2 iteration(s): 3
